In [3]:
import random
import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
from tqdm import tqdm
from progress.bar import IncrementalBar
from IPython.display import clear_output, display

In [4]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
# warnings are not important :)
import warnings
warnings.filterwarnings('ignore')

In [6]:
# import graph module
import sys
sys.path.append("../../")
from tools.graph_tools import *
from oracles.minimization import *
from methods.gradient_tracking import *

In [7]:
def calc_delta_Fx(oracles, x):
    res = []
    for i in range(len(oracles)):
        res.append( oracles[i].grad(np.array([x[i,j] for j in range(2)])) )
    return np.matrix(res)

def calc_F(oracles, x):
    res = []
    for i in range(len(oracles)):
        res.append( oracles[i].func(np.array([x[i,j] for j in range(2)])) )
    return np.matrix(res)

def calc_error(oracles, x_curr, x_prev):
    return np.sum( abs(calc_F(oracles, x_curr) - calc_F(oracles, x_prev)) )  / len(oracles)

def _calc_error(oracles, x_curr, x_prev):
    return np.sum( abs(x_curr - np.array([1, 2])) ) / len(oracles)


In [8]:
class Gradient_track:
    def __init__(self,
            F : list,
            f : list,
            calc_delta_Fx,
            calc_error,
            W : np.matrix,
            x0 : np.matrix,
            xx : np.matrix,
            alpha : float,
            theta : float,
            mu : float,
            err : float,
            max_iter : int = 100,
            need_log : bool = False,
            need_err_logs : bool = False):
        self.F = F
        self.f = f
        self.calc_delta_Fx = calc_delta_Fx
        self.calc_error = calc_error
        self.W = W
        self.x0 = x0
        self.xx = xx
        self.N = len(x0)
        self.alpha = alpha
        self.theta = theta
        self.mu    = mu
        self.err = err
        self.max_iter = max_iter

    def pre_work(self):
        self.y  = x0.copy()
        self.y0 = x0.copy()
        self.z  = x0.copy()
        self.s  = calc_delta_Fx(self.f, self.x0)
        self.x_prev = x0.copy()
        self.x  = self.W * self.z + (1-self.theta) * self.W * self.x0
        self.k  = 0
        self.start_err = self._calc_error()

    def step(self):
        self.k += 1
        self.y0 = self.y.copy()
        self.y  = self.theta * self.z + (1 - self.theta) * self.x
        self.s  = self.W * self.s + \
                  self.calc_delta_Fx(self.f, self.y) - \
                  self.calc_delta_Fx(self.f, self.y0)
        self.x_prev = self.x.copy()
        self.z  = (1/(1+self.mu*self.alpha/self.theta)) *\
                  ( self.W * ((self.mu * self.alpha / self.theta) * self.y + self.z) -\
                   self.alpha / self.theta * self.s )
        self.x  = self.W * self.z + (1-self.theta) * self.W * self.x
        
    #def _calc_error(self):
    #    return self.calc_error(self.F, self.x, self.x0)

    #def _calc_error(self):
    #    return np.sum(abs(self.x - self.xx)) / self.N

    def _calc_error(self):
        return np.sum(abs(self.x - self.x_prev)) / self.N

    def run(self):
        while self._calc_error() > self.err and self.k < self.max_iter:
            self.step()
    
    def run_with_status_bar(self):
        error_bar = np.flip(np.arange(self.err, self.start_err, self.err+ self.start_err / 100))
        print(error_bar)
        i = 0
        curr_error = self._calc_error()
        self.print_bar(0, curr_error)
        while curr_error > self.err and self.k < self.max_iter:
            self.step()
            curr_error = self._calc_error()
            if curr_error < error_bar[i]:
                i+=1
                self.print_bar(i, curr_error)
        self.print_bar(i, curr_error)
        print("\nEnd: ", curr_error)
            
                
    def print_bar(self, i, curr_error):
        clear_output(wait=True)
        print(self.start_err, '[', end='')
        for j in range(i+1):
            print('|', end='')
        for j in range(100-i-1):
            print('-', end='')
        print(']', self.err)
        print('err:\t', curr_error)
        print('k:\t', self.k)
        print('x[0]:\t', self.x[0])
        
    def find_theta_mu(self, alpha, num_of_theta, num_of_mu):
        res = {}
        for theta, mu in tqdm( itertools.product(np.linspace(0.0, 1.0, num_of_theta), np.linspace(0.0, 1.0, num_of_mu)) ):
            self.theta = theta
            self.mu    = mu
            self.pre_work()
            self.run()
            res[theta, mu] = self.k
        {k: v for k, v in sorted(res.items(), key=lambda item: item[1])}
        return res

In [9]:
N = 10
x = np.array([1, 2])
oracles = []
for i in range(N):
    A = np.random.random((2, 2))
    b = A.dot(x)
    oracles.append(LinearRegressionL2Oracle(A, b, regcoef=0.00001))
    
W = make_random_graph_matrix(N, 0.2)
W = fill_metropolis_weigts(W)
#make_graph_img(W, fig_size=(20, 20))

alpha = pow(10, -3)
theta = 0.5
mu = 0

err = pow(10, -12)
max_iter = pow(10, 5)

x0 = np.full((N, 2), [1.5, 1.5])
xx = np.full((N, 2), [1.0, 2.0])

#x0 = np.full((N, 2), [3, 3])

In [10]:
gt = Gradient_track(F = oracles,
                 f = oracles,
                 calc_delta_Fx = calc_delta_Fx,
                 calc_error = calc_error,
                 W = W,
                 x0 = x0,
                 xx = xx,
                 alpha = alpha,
                 theta = theta,
                 mu = mu,
                 err = err,
                 max_iter = max_iter,
                 need_log = False,
                 need_err_logs = True)

gt.pre_work()

In [11]:
gt.find_theta_mu(0.01, 10, 10)

100it [20:59, 12.59s/it]


{(0.0, 0.0): 1,
 (0.0, 0.1111111111111111): 1,
 (0.0, 0.2222222222222222): 1,
 (0.0, 0.3333333333333333): 1,
 (0.0, 0.4444444444444444): 1,
 (0.0, 0.5555555555555556): 1,
 (0.0, 0.6666666666666666): 1,
 (0.0, 0.7777777777777777): 1,
 (0.0, 0.8888888888888888): 1,
 (0.0, 1.0): 1,
 (0.1111111111111111, 0.0): 4518,
 (0.1111111111111111, 0.1111111111111111): 100000,
 (0.1111111111111111, 0.2222222222222222): 82421,
 (0.1111111111111111, 0.3333333333333333): 49354,
 (0.1111111111111111, 0.4444444444444444): 36090,
 (0.1111111111111111, 0.5555555555555556): 28892,
 (0.1111111111111111, 0.6666666666666666): 24351,
 (0.1111111111111111, 0.7777777777777777): 21212,
 (0.1111111111111111, 0.8888888888888888): 18905,
 (0.1111111111111111, 1.0): 17132,
 (0.2222222222222222, 0.0): 16745,
 (0.2222222222222222, 0.1111111111111111): 100000,
 (0.2222222222222222, 0.2222222222222222): 100000,
 (0.2222222222222222, 0.3333333333333333): 100000,
 (0.2222222222222222, 0.4444444444444444): 100000,
 (0.2222222

In [13]:
gt.theta = 0.11111
gt.mu    = 0.0
gt.pre_work()
gt.run_with_status_bar()

2.666669999999999 [|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||] 1e-12
err:	 9.985567928083584e-13
k:	 4518
x[0]:	 [[1.10965085 2.21902596]]

End:  9.985567928083584e-13
